# Doc2Vec
## 5. Doc2Vec모델을 이용한 단어 시각화 및 클러스터링 (진행중)
- 생성된 Doc2Vec 모델로 부터 Word Vector를 구해 PCA와 TSME를 사용해 시각화를 진행한다.
- 생성된 모델은 총 
- 작성 일시: 2018-06-11
- 수정 일시: 2018-06-11
- 작성자: 부현경 (hyunkyung.boo@gmail.com)
- 참고:
    1) PCA와 TSME
    - https://programmers.co.kr/learn/courses/21/lessons/1698
    - https://towardsdatascience.com/pca-using-python-scikit-learn-e653f8989e60
    - https://www.journaldev.com/19279/python-gensim-word2vec
    - http://bcho.tistory.com/1210
    2) 한글 폰트 설정
    - https://dojang.io/mod/page/view.php?id=1159

In [ ]:
import os
model_path_list = []
for (path, dir, files) in os.walk("D:\\Doc2Vec_model_20180608"):
    for filename in files:
        ext = os.path.splitext(filename)[-1]
        if ext != '.npy':
            model_path_list.append(str("%s\\%s" % (path, filename)))
            
            
# if os.path.isfile('questions-words.txt'):
#     for model in word_models:
#         sections = model.accuracy('questions-words.txt')
#         correct, incorrect = len(sections[-1]['correct']), len(sections[-1]['incorrect'])
#         print('%s: %0.2f%% correct (%d of %d)' % (model, float(correct*100)/(correct+incorrect), correct, correct+incorrect))

In [ ]:
from gensim.models.doc2vec import Doc2Vec


def loadModel(model_path):
    model = Doc2Vec.load(model_path)
    return model


# 모든 단어의 위치 벡터
def getWv(model):
    vocab  = list(model.wv.vocab)
    x = model[vocab]
    return x


# n개의 위치벡터값
# getOpionalWv(model, "김태리", 15)
def getOpionalWv(model, word, n):
    sim_words = model.wv.most_similar(word, topn = n)
    word_vec = []
    # 자기 자신에 대한 유사도는 1이다.
    first_element = word_vec.append(model[word])
    for sw in sim_words:
        word_vec.append(model[sw[0]])
    return word_vec


# n개의 위치벡터값과 각 키워드
# getOpionalWv(model, "김태리", 15)
def getOpionalWv_add_word(model, word, n):
    sim_words = model.wv.most_similar(word, topn = n)
    word_vec = []
    # 자기 자신에 대한 유사도는 1이다.
    first_element = word_vec.append(model[word])
    for sw in sim_words:
        word_vec.append([sw[0], model[sw[0]]])
    return word_vec


# 키워드, 유사도, 위치벡터값
def getOpionalWv_add_sim(model, word, n):
    sim_words = model.wv.most_similar(word, topn = n)
    word_vec = []
    # 자기 자신에 대한 유사도는 1이다.
    first_element = [word, 1, model[word]]
    word_vec.append(first_element)
    for sw in sim_words:
        word_vec.append([sw[0], sw[1], model[sw[0]]])
    return word_vec


# 축소된 차원수를 이용해 그림을 그린다.
# def plot(result):
#     # create a scatter plot of the projection
#     pyplot.scatter(result[:, 0], result[:, 1])
#     words = list(model.wv.vocab)
#     for i, word in enumerate(words):
#         pyplot.annotate(word, xy=(result[i, 0], result[i, 1]))
#     pyplot.show()

In [ ]:
model_list = []
for m in model_path_list:
    model_list.append(loadModel(m))
# print(model_list)

In [ ]:
# 그래프 그릴때 사용하는 패기지
import matplotlib as mpl
import matplotlib.pyplot as plt

# 폰트 설정
mpl.rcParams['axes.unicode_minus'] = False
font_name = mpl.font_manager.FontProperties(fname='C:/Windows/Fonts/malgun.ttf').get_name()
mpl.rc('font', family=font_name)
# print (plt.rcParams['font.family'] )

#### 1. PCA를 사용한 시각화

In [ ]:
from sklearn import decomposition
from sklearn.decomposition import PCA

from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
import numpy as np

cnt = 0
for model in model_list:
    cnt += 1
    vocab = list(model.wv.vocab)
    X = model[vocab]
#     print(len(X))
#     print(X[0][:50])
    pca = PCA(n_components=2)
    X_pca = pca.fit_transform(X[:50, :])
    print(len(X_pca), X_pca)
    df = pd.DataFrame(X_pca, index=vocab[:50], columns=['x', 'y'])
    print(df.shape)
    fig = plt.figure()
    fig.set_size_inches(40, 20)
    # 1,1,1은 subplotd의 그리드 인자를 정수 하나에 다 모아 표현한 것.
    ax = fig.add_subplot(1, 1, 1)
    ax.scatter(df['x'], df['y'])

    for word, pos in df.iterrows():
        ax.annotate(word, pos, fontsize=30)
    plt.show()

#### 2. TSNE을 사용한 시각화

In [ ]:
from gensim.models.doc2vec import Doc2Vec
from sklearn.manifold import TSNE
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
cnt = 0
for model in model_list:
    cnt += 1
    vocab = list(model.wv.vocab)
    X = model[vocab]
    # print(len(X))
    # print(X[0][:10])
    tsne = TSNE(n_components=2)

    X_tsne = tsne.fit_transform(X[:50,:])
    # X_tsne = tsne.fit_transform(X)
    
    df = pd.DataFrame(X_tsne, index=vocab[:50], columns=['x', 'y'])
    print(df.shape)
    
    fig = plt.figure()
    fig.set_size_inches(40, 20)
    # 1,1,1은 subplotd의 그리드 인자를 정수 하나에 다 모아 표현한 것.
    ax = fig.add_subplot(1, 1, 1)
    ax.scatter(df['x'], df['y'])

    for word, pos in df.iterrows():
        ax.annotate(word, pos, fontsize=30)
    plt.show()